## Run Harmonic Regression on Tesseract

In [8]:
import geodesic
import geodesic.tesseract as T
import numpy as np

In [7]:
proj = geodesic.set_active_project("har-reg")

In [5]:
landsat = geodesic.get_dataset("landsat-8")
print(landsat)

dataset:earth-bservation:satellite:electro-optical:landsat-8


In [6]:
bbox = (-94.691162,44.414164,-94.218750,44.676466) # Southern Minnesota

In [ ]:
data_input_step = T.Step(
    name="landsat",
    inputs=[
        T.StepInput(
            dataset="landsat-8",
            project=proj,
            asset_bands=[
                T.AssetBands(
                    asset="SR_B2",
                    bands=[0]
                ),
                T.AssetBands(
                    asset="SR_B3",
                    bands=[0]
                ),
                T.AssetBands(
                    asset="SR_B4",
                    bands=[0]
                ),
                T.AssetBands(
                    asset="SR_B5",
                    bands=[0]
                ),
                T.AssetBands(
                    asset="SR_B6",
                    bands=[0]
                ),
                T.AssetBands(
                    asset="SR_B7",
                    bands=[0]
                ),
            ]
        )
    ],
    outputs=[
        T.StepOutput(
            asset_name="landsat",
            type="tensor",
            chip_size=128,
            pixel_dtype=np.uint16,
            output_time_bins=T.OutputTimeBins(
                
            )
        )
    ]
)